In [1]:
import pandas as pd
import numpy as np

## Get Data

In [2]:
galaxyZoo2 = r'/home/rick/Proyectos/minIA.old/notebooks/DataCharacterization/zoo2MainSpecz.csv'
df_g = pd.read_csv(galaxyZoo2)
df = df_g.sample(n=5000, random_state=1)

In [3]:
classes = [
    'dr7objid',
    't08_odd_feature_a19_ring_weighted_fraction', 
    't08_odd_feature_a20_lens_or_arc_weighted_fraction', 
    't08_odd_feature_a21_disturbed_weighted_fraction', 
    't08_odd_feature_a22_irregular_weighted_fraction', 
    't08_odd_feature_a24_merger_weighted_fraction', 
    't08_odd_feature_a38_dust_lane_weighted_fraction',
    
    't11_arms_number_a31_1_weighted_fraction', 
    't11_arms_number_a32_2_weighted_fraction', 
    't11_arms_number_a33_3_weighted_fraction', 
    't11_arms_number_a34_4_weighted_fraction',
    
    't05_bulge_prominence_a10_no_bulge_weighted_fraction', 
    't05_bulge_prominence_a11_just_noticeable_weighted_fraction', 
    't05_bulge_prominence_a12_obvious_weighted_fraction', 
    't05_bulge_prominence_a13_dominant_weighted_fraction', 
    
    't07_rounded_a16_completely_round_weighted_fraction', 
    't07_rounded_a17_in_between_weighted_fraction', 
    't07_rounded_a18_cigar_shaped_weighted_fraction',
    
    't09_bulge_shape_a25_rounded_weighted_fraction', 
    't09_bulge_shape_a26_boxy_weighted_fraction', 
    't09_bulge_shape_a27_no_bulge_weighted_fraction',
    
    't10_arms_winding_a28_tight_weighted_fraction', 
    't10_arms_winding_a29_medium_weighted_fraction', 
    't10_arms_winding_a30_loose_weighted_fraction']

debiased = [
    'dr7objid',
    't08_odd_feature_a19_ring_debiased',
    't08_odd_feature_a20_lens_or_arc_debiased',
    't08_odd_feature_a21_disturbed_debiased',
    't08_odd_feature_a22_irregular_debiased',
    't08_odd_feature_a24_merger_debiased',
    't08_odd_feature_a38_dust_lane_debiased', 
    
    't11_arms_number_a31_1_debiased', 
    't11_arms_number_a32_2_debiased', 
    't11_arms_number_a33_3_debiased', 
    't11_arms_number_a34_4_debiased',
    't05_bulge_prominence_a10_no_bulge_debiased', 
    't05_bulge_prominence_a11_just_noticeable_debiased', 
    't05_bulge_prominence_a12_obvious_debiased', 
    't05_bulge_prominence_a13_dominant_debiased', 
    
    't07_rounded_a16_completely_round_debiased', 
    't07_rounded_a17_in_between_debiased',
    't07_rounded_a18_cigar_shaped_debiased',
    
    't09_bulge_shape_a25_rounded_debiased', 
    't09_bulge_shape_a26_boxy_debiased', 
    't09_bulge_shape_a27_no_bulge_debiased',
    
    't10_arms_winding_a28_tight_debiased', 
    't10_arms_winding_a29_medium_debiased', 
    't10_arms_winding_a30_loose_debiased']


In [4]:
def classPerImage(df,criteria, threshold):
    df2 = df[criteria].copy()
    df2 = df2[df2 > threshold]
    df2.columns = range(len(df2.columns))
    names = df2[0]
    img_class = (df2.iloc[:, 1:]
                 .stack()
                 .reset_index()
                 .groupby(['level_0'], sort=False)['level_1']
                 .apply(list))
    img = pd.concat([names, img_class], axis=1)
    img.columns = ['id','classes']
    return img

In [5]:
def filterDS(df, criteria, threshold):
    df2 = df[criteria].copy()
    df2 = df2[df2 > threshold]
    df2.columns = range(len(df2.columns))
    return df2

In [7]:
def createDSfile(ruta, df):
    with open(ruta,'w') as dataset:
        dataset.write('type_galaxy_id,images\n')
        classes = list()
        for class_ in range(1, len(df.columns) - 1):
            images_class = df[[0,class_]].dropna()[0].astype(str).tolist()
            images = ' '.join(images_class)
            dataset.write(str(class_)+','+images+'\n')
            classes.append(len(images_class))
    print(classes, sum(classes))

### Clases asignadas utilizando weighted_fraction

In [16]:
img = classPerImage(df_g,classes,0.9)
print('Imágenes con almenos una clase asignada: ', 304122  - img['classes'].isna().sum())
img.head()

Imágenes con almenos una clase asignada:  240589


,id,classes
0,588017703996096547,NaN
1,587738569780428805,"[8, 17, 21]"
2,587735695913320507,[20]
3,587742775634624545,NaN
4,587732769983889439,"[17, 21]"


### Clases asignadas utilizando debiased

In [17]:
img = classPerImage(df_g,debiased,0.9)
print('Imágenes con almenos una clase asignada: ', 304122  - img['classes'].isna().sum())
img.head()

Imágenes con almenos una clase asignada:  240995


,id,classes
0,588017703996096547,NaN
1,587738569780428805,"[8, 17, 21]"
2,587735695913320507,[20]
3,587742775634624545,NaN
4,587732769983889439,"[17, 21]"


### Creación de archivo con elementos por clase

In [18]:
ruta = r'/home/rick/Proyectos/minIA/datasetGZ2.csv'
df_img = filterDS(df_g, classes,0.8) # use df for testing
createDSfile(ruta, df_img)

[12411, 4327, 10165, 7485, 2397, 1385, 3667, 26620, 982, 493, 7064, 18887, 24618, 3662, 38889, 68232, 26260, 55128, 3616, 12923, 28497, 15806] 373514


In [19]:
elemxClass = [12411, 4327, 10165, 7485, 2397, 1385, 3667, 26620, 982, 493, 7064, 18887, 24618, 3662, 38889, 68232, 26260, 55128, 3616, 12923, 28497, 15806] 
for i,j in zip(debiased[1:-1], elemxClass):
    print (j, i)
sum(elemxClass)

12411 t08_odd_feature_a19_ring_debiased
4327 t08_odd_feature_a20_lens_or_arc_debiased
10165 t08_odd_feature_a21_disturbed_debiased
7485 t08_odd_feature_a22_irregular_debiased
2397 t08_odd_feature_a24_merger_debiased
1385 t08_odd_feature_a38_dust_lane_debiased
3667 t11_arms_number_a31_1_debiased
26620 t11_arms_number_a32_2_debiased
982 t11_arms_number_a33_3_debiased
493 t11_arms_number_a34_4_debiased
7064 t05_bulge_prominence_a10_no_bulge_debiased
18887 t05_bulge_prominence_a11_just_noticeable_debiased
24618 t05_bulge_prominence_a12_obvious_debiased
3662 t05_bulge_prominence_a13_dominant_debiased
38889 t07_rounded_a16_completely_round_debiased
68232 t07_rounded_a17_in_between_debiased
26260 t07_rounded_a18_cigar_shaped_debiased
55128 t09_bulge_shape_a25_rounded_debiased
3616 t09_bulge_shape_a26_boxy_debiased
12923 t09_bulge_shape_a27_no_bulge_debiased
28497 t10_arms_winding_a28_tight_debiased
15806 t10_arms_winding_a29_medium_debiased


373514

56735500